In [36]:
import numpy as np

# read data from text files
with open('./data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('./data/labels.txt', 'r') as f:
    labels = f.read()

# type(reviews)
# print(reviews[:1000])
# print()
# print(labels[:20])
# type("hello")
# a="world"
# print(a[:3])
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
#一句一句话构成的list
reviews_splitjuhua_list = all_text.split('\n')
all_text = ' '.join(reviews_splitjuhua_list)
# len(reviews_splitjuhua_list)
# create a list of words
words = all_text.split()
# words[:30]

# feel free to use this import
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab

#独一无二的词+编号
vocab_and_int = {word: ii for ii, word in enumerate(vocab, 1)}
vocab_and_int
## use the dict to tokenize each review in reviews_splitjuhua_list
## store the tokenized reviews in reviews_anzhaojuzi_intsarray
reviews_anzhaojuzi_intsarray = []
for review in reviews_splitjuhua_list:
    reviews_anzhaojuzi_intsarray.append([vocab_and_int[word] for word in review.split()])
print()

In [37]:
# stats about vocabulary
print('Unique words: ', len((vocab_and_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_anzhaojuzi_intsarray[:1])
length=len(reviews_anzhaojuzi_intsarray)
length
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
print(encoded_labels)
# outlier review stats
review_lens = Counter([len(x) for x in reviews_anzhaojuzi_intsarray])
review_lens
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]
[1 0 1 ... 1 0 0]
Zero-length reviews: 1
Maximum review length: 2514


In [38]:
print('Number of reviews before removing outliers: ', len(reviews_anzhaojuzi_intsarray))
## remove any reviews/labels with zero length from the reviews_anzhaojuzi_intsarray list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_anzhaojuzi_intsarray) if len(review) != 0]
len(reviews_anzhaojuzi_intsarray)
# remove 0-length reviews and their labels
reviews_anzhaojuzi_intsarray = [reviews_anzhaojuzi_intsarray[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])
print(len(encoded_labels))
print('Number of reviews after removing outliers: ', len(reviews_anzhaojuzi_intsarray))
len(reviews_anzhaojuzi_intsarray)

Number of reviews before removing outliers:  25001
25000
Number of reviews after removing outliers:  25000


25000

In [39]:
#选择每个句子长为200
seq_len = 200

def pad_features(reviews_anzhaojuzi_intsarray, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's
        or truncated to the input seq_length.
    '''

    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_anzhaojuzi_intsarray), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(reviews_anzhaojuzi_intsarray):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features



# Test your implementation!

seq_length = 200

features = pad_features(reviews_anzhaojuzi_intsarray, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_anzhaojuzi_intsarray), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches
print(features)
#ndarray型的m*n型数字矩阵

[[    0     0     0 ...     8   215    23]
 [    0     0     0 ...    29   108  3324]
 [22382    42 46418 ...   483    17     3]
 ...
 [   49   107    12 ...  1531    56   339]
 [    0     0     0 ...    81    95    38]
 [    0     0     0 ...    11     6  1321]]


In [22]:
import numpy as np
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=1,test_size=0.2,random_state=0)
for train_index,test_index in ss.split(np.array(reviews_anzhaojuzi_intsarray)):
    train_x = features[train_index]
    train_y = encoded_labels[train_index]
    test_x = features[test_index]
    test_y = encoded_labels[test_index]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nTrain_Y set: \t{}".format(train_y.shape),
      "\nTest set: \t\t{}".format(test_x.shape))



			Feature Shapes:
Train set: 		(20000, 200) 
Train_Y set: 	(20000,) 
Test set: 		(5000, 200)


<ipython-input-22-ddaa465405e9>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for train_index,test_index in ss.split(np.array(reviews_ints)):


In [23]:
# a=lambda x:x*x
# print(a(3))
#
# b=lambda x:x*2
# b(20)
#
# dict={1:"the",2:"and"}
#
# students = [('john', 'A', 15), ('jane', 'B', 12), ('dave', 'B', 10)]
# sorted(students, key=lambda s: s[2])

list1=['a','b','c']
list2=[1,2,3]
dict1={a:a for a in list1}
dict1

print([i for i in range(1, 11)])#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

a = [1, 2, 3, 4, 5]
b = ["a", "b", "c", "d", "e"]
# 如何得出c = ["a1", "b2", "c3", "d4", "e5"]
c=[str(x)+str(y) for x,y in zip(b,a)]
c

#if else列表表达式
d=[0 if x%2==0 else x*2 for x in a]
len(d)

np.random.randint(0,100,size=(2,4),dtype=int)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


array([[31,  5, 30, 73],
       [17, 73, 96, 70]])

In [24]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))


			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [25]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


In [26]:
import torch
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [27]:
import torch.nn as nn


class SentimentRNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, bidirectional=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)

        # dropout layer
        self.dropout = nn.Dropout(0.3)#以0.3的概率让某些神经元停止工作

        # linear and sigmoid layers
        if bidirectional:
          self.fc = nn.Linear(hidden_dim*2, output_size)
        else:
          self.fc = nn.Linear(hidden_dim, output_size)

        self.sig = nn.Sigmoid()


    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

#         if bidirectional:
#           lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim*2)
#         else:
#           lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        number = 1
        if self.bidirectional:
           number = 2

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().cuda()
                     )
        else:
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_()
                     )

        return hidden


vocab_size = len(vocab_and_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
bidirectional = False  #这里为True，为双向LSTM

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, bidirectional)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [28]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)
    counter = 0

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.624005... Val Loss: 0.644192
Epoch: 1/4... Step: 200... Loss: 0.687304... Val Loss: 0.688116
Epoch: 1/4... Step: 300... Loss: 0.656812... Val Loss: 0.664682
Epoch: 1/4... Step: 400... Loss: 0.689003... Val Loss: 0.680248
Epoch: 2/4... Step: 100... Loss: 0.578315... Val Loss: 0.617746
Epoch: 2/4... Step: 200... Loss: 0.562378... Val Loss: 0.622129
Epoch: 2/4... Step: 300... Loss: 0.563897... Val Loss: 0.469249
Epoch: 2/4... Step: 400... Loss: 0.437930... Val Loss: 0.443221
Epoch: 3/4... Step: 100... Loss: 0.441925... Val Loss: 0.452530
Epoch: 3/4... Step: 200... Loss: 0.196485... Val Loss: 0.450632
Epoch: 3/4... Step: 300... Loss: 0.242050... Val Loss: 0.429014
Epoch: 3/4... Step: 400... Loss: 0.290332... Val Loss: 0.487103
Epoch: 4/4... Step: 100... Loss: 0.259542... Val Loss: 0.491247
Epoch: 4/4... Step: 200... Loss: 0.111814... Val Loss: 0.470559
Epoch: 4/4... Step: 300... Loss: 0.196392... Val Loss: 0.511773
Epoch: 4/4... Step: 400... Loss: 0.17094

In [30]:
#测试测试测试
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.481
Test accuracy: 0.816


In [29]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = torch.tensor([44, 55, 66, 44, 55, 66, 44, 55, 66, 44, 55, 66])
a
# TensorDataset对tensor进行打包
train_ids = TensorDataset(a, b)
train_ids
for x_train, y_label in train_ids:
    print(x_train, y_label)

# dataloader进行数据封装
print('=' * 80)
train_loader = DataLoader(dataset=train_ids, batch_size=4, shuffle=True)
print(train_loader)
for i, data in enumerate(train_loader, 1):
# 注意enumerate返回值有两个,一个是序号，一个是数据（包含训练数据和标签）
    x_data, label = data
    print(' batch:{0} x_data:{1}  label: {2}'.format(i, x_data, label))


tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
tensor([1, 2, 3]) tensor(44)
tensor([4, 5, 6]) tensor(55)
tensor([7, 8, 9]) tensor(66)
 batch:1 x_data:tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9],
        [4, 5, 6]])  label: tensor([44, 55, 66, 55])
 batch:2 x_data:tensor([[4, 5, 6],
        [1, 2, 3],
        [7, 8, 9],
        [1, 2, 3]])  label: tensor([55, 44, 66, 44])
 batch:3 x_data:tensor([[7, 8, 9],
        [7, 8, 9],
        [1, 2, 3],
        [4, 5, 6]])  label: tensor([66, 66, 44, 55])


In [ ]:
import torch
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np
BATCH_SIZE = 50

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn=torch.nn.LSTM(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        self.out=torch.nn.Linear(in_features=64,out_features=10)

    def forward(self,x):
        # 以下关于shape的注释只针对单向
        # output: [batch_size, time_step, hidden_size]
        # h_n: [num_layers,batch_size, hidden_size] # 虽然LSTM的batch_first为True,但是h_n/c_n的第一维还是num_layers
        # c_n: 同h_n
        output,(h_n,c_n)=self.rnn(x)
        print(output.size())
        # output_in_last_timestep=output[:,-1,:] # 也是可以的
        output_in_last_timestep=h_n[-1,:,:]
        # print(output_in_last_timestep.equal(output[:,-1,:])) #ture
        x=self.out(output_in_last_timestep)
        return x

if __name__ == "__main__":
    # 1. 加载数据
    training_dataset = torchvision.datasets.MNIST("./mnist", train=True,
                                                  transform=torchvision.transforms.ToTensor(), download=True)
    dataloader = Data.DataLoader(dataset=training_dataset,
                                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    # showSample(dataloader)
    test_data=torchvision.datasets.MNIST(root="./mnist",train=False,
                                        transform=torchvision.transforms.ToTensor(),download=False)
    test_dataloader=Data.DataLoader(
        dataset=test_data,batch_size=1000,shuffle=False,num_workers=2)
    testdata_iter=iter(test_dataloader)
    test_x,test_y=testdata_iter.next()
    test_x=test_x.view(-1,28,28)
    # 2. 网络搭建
    net=RNN()
    # 3. 训练
    # 3. 网络的训练（和之前CNN训练的代码基本一样）
    optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
    loss_F=torch.nn.CrossEntropyLoss()
    for epoch in range(1): # 数据集只迭代一次
        for step, input_data in enumerate(dataloader):
            x,y=input_data
            pred=net(x.view(-1,28,28))

            loss=loss_F(pred,y) # 计算loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if step%50==49: # 每50步，计算精度
                with torch.no_grad():
                    test_pred=net(test_x)
                    prob=torch.nn.functional.softmax(test_pred,dim=1)
                    pred_cls=torch.argmax(prob,dim=1)
                    acc=(pred_cls==test_y).sum().numpy()/pred_cls.size()[0]
                    print(f"{epoch}-{step}: accuracy:{acc}")
